In [ ]:

############################################
# 0. 라이브러리 설치 (최초 1회)
############################################

!pip install -U openai pandas scikit-learn tqdm python-dotenv

############################################
# 1. 기본 설정
############################################
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.cluster import KMeans
import asyncio
from datetime import datetime
import openai
from openai import AsyncOpenAI
import logging
from dotenv import load_dotenv
load_dotenv()

# 동기 클라이언트 (임베딩용)
client = openai
openai.api_key = os.getenv("OPENAI_API_KEY")

# 비동기 클라이언트 (채팅 완료용 - 속도 개선)
aclient = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

if not openai.api_key:
    logging.error("OpenAI API 키가 설정되어 있지 않습니다.")
    # exit(1) # 노트북에서는 커널이 죽을 수 있으므로 주석 처리


In [ ]:

############################################
# 2. CSV 로드
############################################
FILE_PATH = r"D:\git_rk\data\survey.csv"
TEXT_COL = "answer"

if not os.path.exists(FILE_PATH):
    print(f"파일을 찾을 수 없습니다: {FILE_PATH}")
else:
    df = pd.read_csv(FILE_PATH)
    # 데이터 전처리
    if TEXT_COL in df.columns:
        df = df[df[TEXT_COL].notna()].copy()
        df.rename(columns={TEXT_COL: "원본"}, inplace=True)
        texts = df["원본"].astype(str).tolist()
        print(f"총 {len(texts)}건 주관식 응답 로드 완료")
    else:
        print(f"컬럼 '{TEXT_COL}'이 데이터에 없습니다. 컬럼명을 확인하세요.")
        print(f"현재 컬럼: {df.columns.tolist()}")


In [ ]:

############################################
# 3. 원본 → 요약 (Async 최적화)
############################################

async def summarize_row_async(text, sem, pbar):
    prompt = f"""
다음은 축제(행사)에 대한 설문 주관식 응답입니다.

⚠️ 매우 중요:
- 원문에 없는 내용은 절대 작성하지 마세요.
- 추론, 일반화, 의미 확장 금지
- '요약', '요약:' 같은 라벨이나 제목을 절대 출력하지 마세요.
- 출력은 내용만 작성하세요.

아래 응답에서
응답자가 직접 언급한 핵심 내용만
한 줄로 자연스럽게 정리해 주세요.

설문 응답:
{text}

출력:
"""
    async with sem:
        try:
            r = await aclient.chat.completions.create(
                model="gpt-5-mini",
                messages=[
                    {"role": "system", "content": "설문 응답 요약 전문가"},
                    {"role": "user", "content": prompt}
                ]
            )
            result = r.choices[0].message.content.strip()
        except Exception as e:
            logging.error(f"Error summarizing: {e}")
            result = ""
        finally:
            pbar.update(1)
            return result

async def run_step3():
    # 동시 실행 제한 (Rate Limit 방지)
    sem = asyncio.Semaphore(20)
    pbar = tqdm(total=len(texts), desc="요약 생성 중")
    
    tasks = [summarize_row_async(t, sem, pbar) for t in texts]
    results = await asyncio.gather(*tasks)
    
    pbar.close()
    return results

# 주피터 노트북 환경에서는 await를 직접 사용 가능
if 'texts' in locals() and texts:
    summaries = await run_step3()
    df["요약"] = summaries
    display(df.head())
else:
    print("데이터가 로드되지 않았습니다.")


In [ ]:

############################################
# 4. 요약 → 요약 개조식 (Async 최적화)
############################################

async def bulletize_summary_async(summary, sem, pbar):
    prompt = f"""
아래 문장은 설문 응답 요약 문장입니다.

⚠️ 규칙:
- 아래 문장에 등장한 단어와 표현만 사용
- 새로운 단어, 의미, 해석 추가 금지
- 단어 수는 최대 7개 이내
- 조사 최소화
- 라벨(요약, 정리 등) 출력 금지
- 개조식 한 줄로만 출력

요약 문장:
{summary}

출력:
- 
"""
    async with sem:
        try:
            r = await aclient.chat.completions.create(
                model="gpt-5-mini",
                messages=[
                    {"role": "system", "content": "설문 요약 축약 전문가"},
                    {"role": "user", "content": prompt}
                ]
            )
            result = r.choices[0].message.content.strip()
        except Exception as e:
            logging.error(f"Error bulletizing: {e}")
            result = ""
        finally:
            pbar.update(1)
            return result

async def run_step4():
    sem = asyncio.Semaphore(20)
    if "요약" not in df.columns:
        print("요약 컬럼이 없습니다.")
        return []
        
    input_summaries = df["요약"].tolist()
    pbar = tqdm(total=len(input_summaries), desc="개조식 변환 중")
    
    tasks = [bulletize_summary_async(s, sem, pbar) for s in input_summaries]
    results = await asyncio.gather(*tasks)
    
    pbar.close()
    return results

if 'df' in locals() and "요약" in df.columns:
    bullet_summaries = await run_step4()
    df["요약 개조식"] = bullet_summaries
    display(df.head())


In [ ]:

############################################
# 5. 요약 → 카테고리화 (임베딩 + 클러스터링)
############################################
def get_embeddings_batch(texts, model="text-embedding-3-small", batch_size=50):
    all_embeddings = []
    # 빈 텍스트 제거 또는 처리 로직 추가 권장
    valid_texts = [t if t else " " for t in texts]
    
    for i in tqdm(range(0, len(valid_texts), batch_size), desc="임베딩 생성"):
        batch = valid_texts[i:i + batch_size]
        try:
            r = client.embeddings.create(model=model, input=batch)
            all_embeddings.extend([d.embedding for d in r.data])
        except Exception as e:
            logging.error(f"Embedding Error at batch {i}: {e}")
            
    return np.array(all_embeddings)

if 'df' in locals() and "요약" in df.columns:
    summary_embeddings = get_embeddings_batch(df["요약"].tolist())

    N_CLUSTERS = 5
    # 데이터 개수가 클러스터 수보다 적을 경우 예외 처리
    if len(df) < N_CLUSTERS:
        N_CLUSTERS = len(df)
        print(f"⚠️ 데이터 개수가 적어 클러스터 수를 {N_CLUSTERS}개로 조정합니다.")

    if N_CLUSTERS > 0:
        kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42)
        df["cluster_id"] = kmeans.fit_predict(summary_embeddings)
    else:
        print("클러스터링할 데이터가 없습니다.")


In [ ]:

############################################
# 6. 클러스터 → 카테고리명 생성
############################################
def generate_cluster_name(texts):
    sample = "\n".join(texts[:10])
    prompt = f"""
다음 요약 문장 묶음을 대표하는 카테고리명을 만들어 주세요.

조건:
- 요약 문장에 실제 등장한 표현 기반
- 추론·확장 금지
- 10자 내외 명사형

요약 문장:
{sample}

출력:
"""
    try:
        r = client.chat.completions.create(
            model="gpt-5-mini",
            messages=[
                {"role": "system", "content": "설문 데이터 분류 전문가"},
                {"role": "user", "content": prompt}
            ]
        )
        return r.choices[0].message.content.strip()
    except Exception as e:
        logging.error(f"Cluster Naming Error: {e}")
        return "기타"

if 'df' in locals() and "cluster_id" in df.columns:
    cluster_names = {}
    unique_clusters = sorted(df["cluster_id"].unique())

    print("카테고리명 생성 중...")
    for c in tqdm(unique_clusters):
        texts_c = df[df["cluster_id"] == c]["요약"].tolist()
        cluster_names[c] = generate_cluster_name(texts_c)

    df["카테고리"] = df["cluster_id"].map(cluster_names)


In [ ]:

############################################
# 7. 최종 산출물 저장
############################################
if 'df' in locals():
    base_filename = os.path.splitext(os.path.basename(FILE_PATH))[0]
    now_str = datetime.now().strftime("%Y%m%d_%H%M")

    excel_path = f"data/{base_filename}_{now_str}.xlsx"
    csv_path   = f"data/{base_filename}_{now_str}.csv"

    final_df = df[["원본", "요약", "요약 개조식", "카테고리"]]
    display(final_df.head())

    # data 폴더가 없으면 생성
    os.makedirs("data", exist_ok=True)

    final_df.to_excel(excel_path, index=False)
    final_df.to_csv(csv_path, index=False, encoding="utf-8-sig")

    print(f"\n✅ 엑셀 저장 완료: {excel_path}")
    print(f"✅ CSV 저장 완료: {csv_path}")
